# Prepping data 

In [2]:
# importing libraries
import requests
import re
from bs4 import BeautifulSoup, Comment
import pandas as pd
import numpy as np
import time

In [3]:
# loading dataframe 
df = pd.read_csv('https://raw.githubusercontent.com/michaelseman/Portfolio/main/PythonML_NFL_Draft/BioPlusConfBMI.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7576 entries, 0 to 7575
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   year               7576 non-null   int64  
 1   round              7576 non-null   int64  
 2   draft_pick         7576 non-null   int64  
 3   draft_team         7576 non-null   object 
 4   player             7576 non-null   object 
 5   player_url         7100 non-null   object 
 6   position           7576 non-null   object 
 7   age                7095 non-null   float64
 8   college            7560 non-null   object 
 9   college_stats_url  5748 non-null   object 
 10  pic_url            4122 non-null   object 
 11  bday               7095 non-null   object 
 12  hs                 6808 non-null   object 
 13  height             7088 non-null   object 
 14  weight             7088 non-null   float64
 15  height_decimal     7088 non-null   float64
 16  birthplace_town    6713 

In [5]:
# creating list of defensive positions
defense = ['DE','CB','DT','LB','DB','OLB','DL','ILB']

In [6]:
# creating a new dataframe with only defensive players
df_def = df[df['position'].isin(defense)]

In [7]:
df_def.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3605 entries, 0 to 7572
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   year               3605 non-null   int64  
 1   round              3605 non-null   int64  
 2   draft_pick         3605 non-null   int64  
 3   draft_team         3605 non-null   object 
 4   player             3605 non-null   object 
 5   player_url         3414 non-null   object 
 6   position           3605 non-null   object 
 7   age                3411 non-null   float64
 8   college            3597 non-null   object 
 9   college_stats_url  2704 non-null   object 
 10  pic_url            1880 non-null   object 
 11  bday               3411 non-null   object 
 12  hs                 3304 non-null   object 
 13  height             3407 non-null   object 
 14  weight             3407 non-null   float64
 15  height_decimal     3407 non-null   float64
 16  birthplace_town    3264 

In [8]:
df_def['position'].unique()

array(['DE', 'CB', 'DT', 'LB', 'DB', 'OLB', 'DL', 'ILB'], dtype=object)

In [9]:
# dropping all rows where college_stats_url is null
df_def = df_def.dropna(subset=['college_stats_url'])

In [10]:
df_def.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2704 entries, 0 to 7565
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   year               2704 non-null   int64  
 1   round              2704 non-null   int64  
 2   draft_pick         2704 non-null   int64  
 3   draft_team         2704 non-null   object 
 4   player             2704 non-null   object 
 5   player_url         2612 non-null   object 
 6   position           2704 non-null   object 
 7   age                2610 non-null   float64
 8   college            2704 non-null   object 
 9   college_stats_url  2704 non-null   object 
 10  pic_url            1654 non-null   object 
 11  bday               2610 non-null   object 
 12  hs                 2526 non-null   object 
 13  height             2606 non-null   object 
 14  weight             2606 non-null   float64
 15  height_decimal     2606 non-null   float64
 16  birthplace_town    2484 

In [11]:
df_test = df_def.head(5).copy()

In [12]:
df_test.head()

,year,round,draft_pick,draft_team,player,player_url,position,age,college,college_stats_url,...,bday,hs,height,weight,height_decimal,birthplace_town,birthplace_state,home_state,conference,bmi
0,2022,1,1,JAX,Travon Walker,https://www.pro-football-reference.com/players...,DE,21.0,Georgia,http://www.sports-reference.com/cfb/players/tr...,...,12/18/2000,"Upson,GA",6'5,275.0,6.42,Thomaston,"GA,",GA,SEC,32.6
1,2022,1,2,DET,Aidan Hutchinson,https://www.pro-football-reference.com/players...,DE,22.0,Michigan,http://www.sports-reference.com/cfb/players/ai...,...,8/9/2000,"Divine Child,MI",6'7,265.0,6.58,MI,NaN,MI,BIG10,29.9
2,2022,1,3,HOU,Derek Stingley Jr.,https://www.pro-football-reference.com/players...,CB,21.0,LSU,http://www.sports-reference.com/cfb/players/de...,...,6/20/2001,"Dunham,LA",6'1,195.0,6.08,Baton Rouge,"LA,",LA,SEC,25.8
3,2022,1,4,NYJ,Sauce Gardner,https://www.pro-football-reference.com/players...,CB,22.0,Cincinnati,http://www.sports-reference.com/cfb/players/ah...,...,8/31/2000,"Martin Luther King,MI",6'3,200.0,6.25,Detroit,"MI,",MI,AAC,25.0
4,2022,1,5,NYG,Kayvon Thibodeaux,https://www.pro-football-reference.com/players...,DE,21.0,Oregon,http://www.sports-reference.com/cfb/players/ka...,...,12/15/2000,"Susan Miller Dorsey,CA,Oaks Christian,CA",6'5,250.0,6.42,CA,NaN,CA,PAC12,29.6


# Function: take_url_return_page

In [13]:
def take_url_return_page(url):
  # this function takes a url and returns the soup content for it
  r = requests.get(url)
  page = BeautifulSoup(r.content, 'html.parser')
  return page

# Function: take_page_return_tables

In [14]:
def take_page_return_tables2(page):
  # this function takes a soup page and looks for the table id defense and returns it.  same for the punt_ret table (which is hidden in the comments)
  def_table = page.find('table', {'id': 'defense'})
  # finding where punt return info is located
  all_punt_ret = page.find(id="all_punt_ret")
  # adding an exception.  If all_punt_ret is empty, we return 0
  try:
    # getting the info inside the comments
    punt_ret = all_punt_ret(text=lambda text: isinstance(text, Comment))[0]
    # making the data parsable by BS
    punt_soup=BeautifulSoup(punt_ret, 'html.parser')
    punt_table = punt_soup.find('table', {'id': 'punt_ret'})
    return def_table, punt_table
  except TypeError:
    return def_table, 0

# Function: get_stats

In [15]:
def get_stats(table, columns):
  # this function takes the stats table and returns a dataframe of the stats
  
  # define the DataFrame
  df = pd.DataFrame(columns=columns)  

  # finds all rows in the table
  try:
    rows = table.find_all('tr')
  
    

    # this counter will be used along with total_games to get the total games played in the final row of the dataframe and to index the dataframe to exit the loop.
    row_counter = 0
    total_games = 0

    # loop over each row in the table, skipping over header rows (0,1)
    for row in rows[2:]:
      # creating a list where we will store the stats for each row.  We will then append this list to our df
      stat_list = []
      
      if row.find('th').text == 'Career':
        stat_list.append(row.find('th').text)
      else:
      # this part is for the first rows of our table.  these rows contain the year in an 'a' tag
        try: 
          stat_list.append(row.find('a').text)
        except AttributeError:
          # after year values, we find the text 'Career' inside a th tag.
          try:
            stat_list.append(row.find('th').text)
          # if neither 'a' nor 'th' tags are present
          except AttributeError:
            break
          
      # cells will contain all the data inside our table td tags
      cells = row.find_all('td')
      
      # counter to be used for total games
      cell_counter=0
        
      # loop for pulling all the relevant stats from cells
      for cell in cells:
        # when the cell count is 4, that cell contains the games played... we want to total this for use in the last row
        if cell_counter == 4:
          games = cell.text
          # try/except for catch nulls error
          try:
            games = int(games)
            total_games += games
          except ValueError:
            total_games += 0
          stat_list.append(games)
          cell_counter +=1
        
        # this is for all the other cells
        else:
          stat_list.append(cell.text)
          cell_counter +=1
      
      df = df.append(pd.DataFrame([stat_list], columns=columns))
      # this is our break to stop the issue when a player has multiple colleges and they are tallied AFTER career
      if df.iloc[row_counter,0]=='Career':
        for i in range(2,5):
          df.iloc[row_counter,i]= df.iloc[row_counter-1,i]
        df.iloc[row_counter,5] = total_games
        break
      row_counter+=1
  except AttributeError:
    df = pd.DataFrame()
  return df

# Define columns

In [16]:
def_columns = ['year', 'school', 'conference', 'class', 'position', 'games', 'solo_tackles', 'assisted_tackles', 'total_tackles', 'tackles_loss', 'sacks', 'interceptions', 'int_yards', 'int_yards_per_int', 'int_td', 'passes_defended', 'fumbles_recovered', 'fumble_yards', 'fumble_td', 'forced_fumbles']
punt_columns = ['year','school','conference','class','position','games','punt_returns','punt_ret_yds','punt_ret_avg','punt_td','kick_returns','kick_ret_yds','kick_ret_avg','kick_ret_td']
pass_columns = ['year','school','conference','class','position','games','completions','attempts','comp_pct','passing_yds','yds_per_att','adj_yds_per_att','passing_td','interceptions','pass_eff_rate']
rush_columns = ['year','school','conference','class','position','games','rush_attempts','rush_yds','rush_avg','rush_td','receptions','rec_yds','rec_avg','rec_td','total_plays','total_yds','total_avg','total_td']
rec_columns = ['year','school','conference','class','position','games','receptions','rec_yds','rec_avg','rec_td','rush_attempts','rush_yds','rush_avg','rush_td','total_plays','total_yds','total_avg','total_td']

# Testing Phase

In [ ]:
df_test = df_def.head(10).copy()
df_test = df_test.reset_index(drop=True)

In [ ]:
len(df_test)

10

In [ ]:
url = df_test.loc[0,'college_stats_url']

In [ ]:
print(url)

http://www.sports-reference.com/cfb/players/travon-walker-1.html


In [ ]:
page = take_url_return_page(df_test.iloc[0,9])
page.find('table')

<table class="sortable stats_table" data-cols-to-freeze=",1" id="defense">
<caption>Defense &amp; Fumbles Table</caption>
<colgroup><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/></colgroup>
<thead>
<tr class="over_header">
<th aria-label="" class="over_header center" colspan="5" data-stat=""></th><th></th>
<th aria-label="" class="over_header center" colspan="5" data-stat="header_tackles">Tackles</th>
<th aria-label="" class="over_header center" colspan="5" data-stat="header_def_int">Def Int</th>
<th aria-label="" class="over_header center" colspan="4" data-stat="header_fumbles">Fumbles</th>
</tr>
<tr>
<th aria-label="Year" class="poptip sort_default_asc right" data-stat="year_id" scope="col">Year</th>
<th aria-label="School" class="poptip sort_default_asc left" data-stat="school_name" data-tip="School Name" scope="col">School</th>
<th aria-label="Conf" class="poptip sort_default_asc left" data-stat="conf_abbr" s

In [ ]:
page = take_url_return_page(df_test.iloc[0,9])
def_table, punt_table = take_page_return_tables(page)
stat_df = get_stats(def_table, def_columns)

<ipython-input-14-aa696c57920a>:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([stat_list], columns=columns))
<ipython-input-14-aa696c57920a>:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([stat_list], columns=columns))
<ipython-input-14-aa696c57920a>:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([stat_list], columns=columns))
<ipython-input-14-aa696c57920a>:58: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([stat_list], columns=columns))


In [ ]:
stat_df

,year,school,conference,class,position,games,solo_tackles,assisted_tackles,total_tackles,tackles_loss,sacks,interceptions,int_yards,int_yards_per_int,int_td,passes_defended,fumbles_recovered,fumble_yards,fumble_td,forced_fumbles
0,2019,Georgia,SEC,JR,LB,9,9,11,20,2.0,1.5,0,0,,0,1,0,,,1
0,2020,Georgia,SEC,SR,LB,7,11,5,16,5.0,5.0,0,0,,0,1,0,,,0
0,2021,Florida State,ACC,SR,DE,12,33,37,70,17.5,11.5,0,0,,0,2,1,,1,2
0,Career,Overall,ACC,SR,DE,28,53,53,106,24.5,18.0,0,0,,0,4,1,,1,3


In [ ]:
subset_df = stat_df.iloc[:-1, :]
row_to_add = None

for j in range(len(subset_df)-1, -1, -1):
    row = subset_df.iloc[j]
    games_str = row['games']
    if games_str:
        games = int(games_str)
    else:
        games = 0 # or some other default value
    if games >= 6:
        row_to_add = row
        print(row)
        break

if row_to_add is None:
    row_to_add = subset_df.iloc[-1]

# add the string 'Best_' to all the column names in row_to_add
prefix = 'Best_'
updated_columns = [prefix + col for col in row_to_add.index]
row_to_add.columns = updated_columns

df_test.loc[0, updated_columns] = row_to_add.values

year                    2021
school               Georgia
conference               SEC
class                     JR
position                  DL
games                     13
solo_tackles              17
assisted_tackles          16
total_tackles             33
tackles_loss             7.5
sacks                    6.0
interceptions              0
int_yards                  0
int_yards_per_int           
int_td                     0
passes_defended            2
fumbles_recovered          1
fumble_yards                
fumble_td                   
forced_fumbles             0
Name: 0, dtype: object


In [ ]:
df_test

,year,round,draft_pick,draft_team,player,player_url,position,age,college,college_stats_url,pic_url,bday,hs,height,weight,height_decimal,birthplace_town,birthplace_state,home_state,conference,bmi,returner,Career_year,Career_school,Career_conference,Career_class,Career_position,Career_games,Career_solo_tackles,Career_assisted_tackles,Career_total_tackles,Career_tackles_loss,Career_sacks,Career_interceptions,Career_int_yards,Career_int_yards_per_int,Career_int_td,Career_passes_defended,Career_fumbles_recovered,Career_fumble_yards,Career_fumble_td,Career_forced_fumbles,Best_year,Best_school,Best_conference,Best_class,Best_position,Best_games,Best_solo_tackles,Best_assisted_tackles,Best_total_tackles,Best_tackles_loss,Best_sacks,Best_interceptions,Best_int_yards,Best_int_yards_per_int,Best_int_td,Best_passes_defended,Best_fumbles_recovered,Best_fumble_yards,Best_fumble_td,Best_forced_fumbles
0,2022,1,1,JAX,Travon Walker,https://www.pro-football-reference.com/players...,DE,21.0,Georgia,http://www.sports-reference.com/cfb/players/tr...,https://www.pro-football-reference.com/req/201...,12/18/2000,"Upson,GA",6'5,275.0,6.42,Thomaston,"GA,",GA,SEC,32.6,0.0,Career,Georgia,SEC,JR,DL,29.0,32,29,61,13.0,9.5,1,0,0.0,0,3,1,,,1,2021,Georgia,SEC,JR,DL,13,17,16,33,7.5,6.0,0,0,,0,2,1,,,0


In [ ]:
df_test

,year,round,draft_pick,draft_team,player,player_url,position,age,college,college_stats_url,pic_url,bday,hs,height,weight,height_decimal,birthplace_town,birthplace_state,home_state,conference,bmi,returner,Career_year,Career_school,Career_conference,Career_class,Career_position,Career_games,Career_solo_tackles,Career_assisted_tackles,Career_total_tackles,Career_tackles_loss,Career_sacks,Career_interceptions,Career_int_yards,Career_int_yards_per_int,Career_int_td,Career_passes_defended,Career_fumbles_recovered,Career_fumble_yards,Career_fumble_td,Career_forced_fumbles,Best_year,Best_school,Best_conference,Best_class,Best_position,Best_games,Best_solo_tackles,Best_assisted_tackles,Best_total_tackles,Best_tackles_loss,Best_sacks,Best_interceptions,Best_int_yards,Best_int_yards_per_int,Best_int_td,Best_passes_defended,Best_fumbles_recovered,Best_fumble_yards,Best_fumble_td,Best_forced_fumbles
0,2022,1,1,JAX,Travon Walker,https://www.pro-football-reference.com/players...,DE,21.0,Georgia,http://www.sports-reference.com/cfb/players/tr...,https://www.pro-football-reference.com/req/201...,12/18/2000,"Upson,GA",6'5,275.0,6.42,Thomaston,"GA,",GA,SEC,32.6,0.0,Career,Georgia,SEC,JR,DL,29.0,32,29,61,13.0,9.5,1,0,0.0,0,3,1,,,1,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021,2021


In [ ]:
row_to_add

Best_year                 2020
Best_school                LSU
Best_conference            SEC
Best_class                  SO
Best_position               CB
Best_games                   7
Best_solo_tackles           19
Best_assisted_tackles        8
Best_total_tackles          27
Best_tackles_loss          2.5
Best_sacks                 0.0
Best_interceptions           0
Best_int_yards               0
Best_int_yards_per_int        
Best_int_td                  0
Best_passes_defended         5
Best_fumbles_recovered       1
Best_fumble_yards             
Best_fumble_td                
Best_forced_fumbles          1
Name: 0, dtype: object

In [ ]:
# getting career row
row_to_add = stat_df[stat_df['year']=='Career']
# changing career row column names
prefix = str(row_to_add.loc[0,'year']) + '_'
updated_columns = [prefix + col for col in def_columns]
row_to_add.columns = updated_columns

temp_df = pd.DataFrame(df_test.iloc[0]).T

# adding career row
df = pd.concat([temp_df, row_to_add], axis=1)

# adding punt return
if punt_table == 0:
  df.at[0, 'returner'] = 0
else:
  df.at[0, 'returner'] = 1

In [ ]:
df

,year,round,draft_pick,draft_team,player,player_url,position,age,college,college_stats_url,...,Career_sacks,Career_interceptions,Career_int_yards,Career_int_yards_per_int,Career_int_td,Career_passes_defended,Career_fumbles_recovered,Career_fumble_yards,Career_fumble_td,Career_forced_fumbles
0,2022,1,1,JAX,Travon Walker,https://www.pro-football-reference.com/players...,DE,21.0,Georgia,http://www.sports-reference.com/cfb/players/tr...,...,9.5,1,0,0.0,0,3,1,,,1


first loop attempt

In [ ]:
columns = df.columns

In [ ]:
final_df = pd.DataFrame(columns=columns)

In [ ]:
df_test.loc[1,'college_stats_url']

'http://www.sports-reference.com/cfb/players/aidan-hutchinson-1.html'

In [ ]:
for i in range(len(df_test)):
  # get the page and tables for this row
  page = take_url_return_page(df_test.iloc[i,9])
  def_table, punt_table = take_page_return_tables(page)

  # get the defensive stats and update the column names
  stat_df = get_stats(def_table, def_columns)
  row_to_add = stat_df[stat_df['year'] == 'Career']
  prefix = str(row_to_add.loc[0, 'year']) + '_'
  updated_columns = [prefix + col for col in def_columns]
  row_to_add.columns = updated_columns

  # add the punt return information to the original DataFrame
  if punt_table == 0:
    df_test.at[i, 'returner'] = 0
  else:
    df_test.at[i, 'returner'] = 1

  # update the original row with the new Career row
  df_test.loc[i, updated_columns] = row_to_add.iloc[0]
  print(df_test.loc[i])
  
  subset_df = stat_df.iloc[:-1, :]
  row_to_add = None

  for j in range(len(subset_df)-1, -1, -1):
      row = subset_df.iloc[j]
      games_str = row['games']
      if games_str:
          games = int(games_str)
      else:
          games = 0 # or some other default value
      if games >= 6:
          row_to_add = row
          break

  if row_to_add is None:
      row_to_add = subset_df.iloc[-1]

  # add the string 'Best_' to all the column names in row_to_add
  prefix = 'Best_'
  updated_columns = [prefix + col for col in row_to_add.index]
  row_to_add.columns = updated_columns

  df_test.loc[i, updated_columns] = row_to_add.values
  time.sleep(5)




AttributeError: ignored

In [ ]:
df_test.head(10)

,year,round,draft_pick,draft_team,player,player_url,position,age,college,college_stats_url,pic_url,bday,hs,height,weight,height_decimal,birthplace_town,birthplace_state,home_state,conference,bmi,returner,Career_year,Career_school,Career_conference,Career_class,Career_position,Career_games,Career_solo_tackles,Career_assisted_tackles,Career_total_tackles,Career_tackles_loss,Career_sacks,Career_interceptions,Career_int_yards,Career_int_yards_per_int,Career_int_td,Career_passes_defended,Career_fumbles_recovered,Career_fumble_yards,Career_fumble_td,Career_forced_fumbles,Best_year,Best_school,Best_conference,Best_class,Best_position,Best_games,Best_solo_tackles,Best_assisted_tackles,Best_total_tackles,Best_tackles_loss,Best_sacks,Best_interceptions,Best_int_yards,Best_int_yards_per_int,Best_int_td,Best_passes_defended,Best_fumbles_recovered,Best_fumble_yards,Best_fumble_td,Best_forced_fumbles
0,2022,1,1,JAX,Travon Walker,https://www.pro-football-reference.com/players...,DE,21.0,Georgia,http://www.sports-reference.com/cfb/players/tr...,https://www.pro-football-reference.com/req/201...,12/18/2000,"Upson,GA",6'5,275.0,6.42,Thomaston,"GA,",GA,SEC,32.6,0.0,Career,Georgia,SEC,JR,DL,29.0,32,29,61,13.0,9.5,1,0,0.0,0,3,1,,,1,2021,Georgia,SEC,JR,DL,13.0,17,16,33,7.5,6.0,0,0,,0,2,1,,,0
1,2022,1,2,DET,Aidan Hutchinson,https://www.pro-football-reference.com/players...,DE,22.0,Michigan,http://www.sports-reference.com/cfb/players/ai...,https://www.pro-football-reference.com/req/201...,8/9/2000,"Divine Child,MI",6'7,265.0,6.58,MI,NaN,MI,BIG10,29.9,0.0,Career,Michigan,Big Ten,SR,DL,36.0,85,71,156,27.5,17.5,0,0,,0,10,1,,,5,2021,Michigan,Big Ten,SR,DL,14.0,36,26,62,16.5,14.0,0,0,,0,3,1,,,2
2,2022,1,3,HOU,Derek Stingley Jr.,https://www.pro-football-reference.com/players...,CB,21.0,LSU,http://www.sports-reference.com/cfb/players/de...,https://www.pro-football-reference.com/req/201...,6/20/2001,"Dunham,LA",6'1,195.0,6.08,Baton Rouge,"LA,",LA,SEC,25.8,1.0,Career,LSU,SEC,JR,DB,25.0,56,17,73,7.0,0.0,6,17,2.8,0,20,2,,,2,2020,LSU,SEC,SO,CB,7.0,19,8,27,2.5,0.0,0,0,,0,5,1,,,1
3,2022,1,4,NYJ,Sauce Gardner,https://www.pro-football-reference.com/players...,CB,22.0,Cincinnati,http://www.sports-reference.com/cfb/players/ah...,https://www.pro-football-reference.com/req/201...,8/31/2000,"Martin Luther King,MI",6'3,200.0,6.25,Detroit,"MI,",MI,AAC,25.0,0.0,Career,Cincinnati,American,JR,CB,33.0,68,31,99,5.5,3.5,9,103,11.4,2,16,0,,,0,2021,Cincinnati,American,JR,CB,13.0,28,12,40,5.0,3.0,3,7,2.3,0,4,0,,,0
4,2022,1,5,NYG,Kayvon Thibodeaux,https://www.pro-football-reference.com/players...,DE,21.0,Oregon,http://www.sports-reference.com/cfb/players/ka...,https://www.pro-football-reference.com/req/201...,12/15/2000,"Susan Miller Dorsey,CA,Oaks Christian,CA",6'5,250.0,6.42,CA,NaN,CA,PAC12,29.6,1.0,Career,Oregon,Pac-12,SO,DE,30.0,84,42,126,35.5,19.0,0,0,,0,7,0,,,3,2021,Oregon,Pac-12,SO,DE,10.0,35,14,49,12.0,7.0,0,0,,0,1,0,,,2
5,2022,1,13,PHI,Jordan Davis,https://www.pro-football-reference.com/players...,DT,22.0,Georgia,http://www.sports-reference.com/cfb/players/jo...,https://www.pro-football-reference.com/req/201...,1/12/2000,"Mallard Creek,NC",6'6,340.0,6.50,NaN,NaN,NC,SEC,39.3,0.0,Career,Georgia,SEC,SR,DL,41.0,37,53,90,11.5,7.0,0,0,,0,1,0,,,0,2021,Georgia,SEC,SR,DL,14.0,17,15,32,5.0,2.0,0,0,,0,1,0,,,0
6,2022,1,21,KAN,Trent McDuffie,https://www.pro-football-reference.com/players...,CB,21.0,Washington,http://www.sports-reference.com/cfb/players/tr...,https://www.pro-football-reference.com/req/201...,9/13/2000,"Mater Dei,CA,Servite,CA,St. John Bosco,CA",5'11,195.0,5.92,CA,NaN,CA,PAC12,27.2,1.0,Career,Washington,Pac-12,SO,DB,27.0,71,23,94,4.5,1.0,2,29,14.5,0,8,3,,,3,2021,Washington,Pac-12,SO,DB,11.0,25,10,35,4.0,1.0,0,0,,0,6,0,,,0
7,2022,1,22,GNB,Quay Walker,https://www.pro-football-reference.com/players...,LB,22.0,Georgia,http://www.sports-reference.com/cfb/players/qu...,https://www.pro-football-reference.com/req/201...,5/8/2000,"Crisp County,GA",6'4,240.0,6.33,NaN,NaN,GA,SEC,29.2

In [ ]:
pd.set_option('display.max_columns', 100)

In [ ]:
df_test.head()

,year,round,draft_pick,draft_team,player,player_url,position,age,college,college_stats_url,pic_url,bday,hs,height,weight,height_decimal,birthplace_town,birthplace_state,home_state,conference,bmi,returner,Career_year,Career_school,Career_conference,Career_class,Career_position,Career_games,Career_solo_tackles,Career_assisted_tackles,Career_total_tackles,Career_tackles_loss,Career_sacks,Career_interceptions,Career_int_yards,Career_int_yards_per_int,Career_int_td,Career_passes_defended,Career_fumbles_recovered,Career_fumble_yards,Career_fumble_td,Career_forced_fumbles
0,2022,1,1,JAX,Travon Walker,https://www.pro-football-reference.com/players...,DE,21.0,Georgia,http://www.sports-reference.com/cfb/players/tr...,https://www.pro-football-reference.com/req/201...,12/18/2000,"Upson,GA",6'5,275.0,6.42,Thomaston,"GA,",GA,SEC,32.6,0.0,Career,Georgia,SEC,JR,DL,29.0,32,29,61,13.0,9.5,1,0,0.0,0,3,1,,,1
1,2022,1,2,DET,Aidan Hutchinson,https://www.pro-football-reference.com/players...,DE,22.0,Michigan,http://www.sports-reference.com/cfb/players/ai...,https://www.pro-football-reference.com/req/201...,8/9/2000,"Divine Child,MI",6'7,265.0,6.58,MI,NaN,MI,BIG10,29.9,0.0,Career,Michigan,Big Ten,SR,DL,36.0,85,71,156,27.5,17.5,0,0,,0,10,1,,,5
2,2022,1,3,HOU,Derek Stingley Jr.,https://www.pro-football-reference.com/players...,CB,21.0,LSU,http://www.sports-reference.com/cfb/players/de...,https://www.pro-football-reference.com/req/201...,6/20/2001,"Dunham,LA",6'1,195.0,6.08,Baton Rouge,"LA,",LA,SEC,25.8,1.0,Career,LSU,SEC,JR,DB,25.0,56,17,73,7.0,0.0,6,17,2.8,0,20,2,,,2
3,2022,1,4,NYJ,Sauce Gardner,https://www.pro-football-reference.com/players...,CB,22.0,Cincinnati,http://www.sports-reference.com/cfb/players/ah...,https://www.pro-football-reference.com/req/201...,8/31/2000,"Martin Luther King,MI",6'3,200.0,6.25,Detroit,"MI,",MI,AAC,25.0,0.0,Career,Cincinnati,American,JR,CB,33.0,68,31,99,5.5,3.5,9,103,11.4,2,16,0,,,0
4,2022,1,5,NYG,Kayvon Thibodeaux,https://www.pro-football-reference.com/players...,DE,21.0,Oregon,http://www.sports-reference.com/cfb/players/ka...,https://www.pro-football-reference.com/req/201...,12/15/2000,"Susan Miller Dorsey,CA,Oaks Christian,CA",6'5,250.0,6.42,CA,NaN,CA,PAC12,29.6,1.0,Career,Oregon,Pac-12,SO,DE,30.0,84,42,126,35.5,19.0,0,0,,0,7,0,,,3


In [ ]:
for i in range(len(df_test)):
    try:
        page = take_url_return_page(df_test.loc[i, 'college_stats_url'])
    except KeyError as e:
        print(f"KeyError: {e} for row {i}")
        continue

KeyError: 5 for row 5


In [ ]:
df_test.loc[4,'college_stats_url']

'http://www.sports-reference.com/cfb/players/kayvon-thibodeaux-1.html'

In [ ]:
df_test.head(6)

,year,round,draft_pick,draft_team,player,player_url,position,age,college,college_stats_url,pic_url,bday,hs,height,weight,height_decimal,birthplace_town,birthplace_state,home_state,conference,bmi,returner,Career_year,Career_school,Career_conference,Career_class,Career_position,Career_games,Career_solo_tackles,Career_assisted_tackles,Career_total_tackles,Career_tackles_loss,Career_sacks,Career_interceptions,Career_int_yards,Career_int_yards_per_int,Career_int_td,Career_passes_defended,Career_fumbles_recovered,Career_fumble_yards,Career_fumble_td,Career_forced_fumbles
0,2022,1,1,JAX,Travon Walker,https://www.pro-football-reference.com/players...,DE,21.0,Georgia,http://www.sports-reference.com/cfb/players/tr...,https://www.pro-football-reference.com/req/201...,12/18/2000,"Upson,GA",6'5,275.0,6.42,Thomaston,"GA,",GA,SEC,32.6,0.0,Career,Georgia,SEC,JR,DL,29.0,32,29,61,13.0,9.5,1,0,0.0,0,3,1,,,1
1,2022,1,2,DET,Aidan Hutchinson,https://www.pro-football-reference.com/players...,DE,22.0,Michigan,http://www.sports-reference.com/cfb/players/ai...,https://www.pro-football-reference.com/req/201...,8/9/2000,"Divine Child,MI",6'7,265.0,6.58,MI,NaN,MI,BIG10,29.9,0.0,Career,Michigan,Big Ten,SR,DL,36.0,85,71,156,27.5,17.5,0,0,,0,10,1,,,5
2,2022,1,3,HOU,Derek Stingley Jr.,https://www.pro-football-reference.com/players...,CB,21.0,LSU,http://www.sports-reference.com/cfb/players/de...,https://www.pro-football-reference.com/req/201...,6/20/2001,"Dunham,LA",6'1,195.0,6.08,Baton Rouge,"LA,",LA,SEC,25.8,1.0,Career,LSU,SEC,JR,DB,25.0,56,17,73,7.0,0.0,6,17,2.8,0,20,2,,,2
3,2022,1,4,NYJ,Sauce Gardner,https://www.pro-football-reference.com/players...,CB,22.0,Cincinnati,http://www.sports-reference.com/cfb/players/ah...,https://www.pro-football-reference.com/req/201...,8/31/2000,"Martin Luther King,MI",6'3,200.0,6.25,Detroit,"MI,",MI,AAC,25.0,0.0,Career,Cincinnati,American,JR,CB,33.0,68,31,99,5.5,3.5,9,103,11.4,2,16,0,,,0
4,2022,1,5,NYG,Kayvon Thibodeaux,https://www.pro-football-reference.com/players...,DE,21.0,Oregon,http://www.sports-reference.com/cfb/players/ka...,https://www.pro-football-reference.com/req/201...,12/15/2000,"Susan Miller Dorsey,CA,Oaks Christian,CA",6'5,250.0,6.42,CA,NaN,CA,PAC12,29.6,1.0,Career,Oregon,Pac-12,SO,DE,30.0,84,42,126,35.5,19.0,0,0,,0,7,0,,,3
12,2022,1,13,PHI,Jordan Davis,https://www.pro-football-reference.com/players...,DT,22.0,Georgia,http://www.sports-reference.com/cfb/players/jo...,https://www.pro-football-reference.com/req/201...,1/12/2000,"Mallard Creek,NC",6'6,340.0,6.50,NaN,NaN,NC,SEC,39.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Function: the_loop

In [60]:
def the_loop(df_test):
  for i in range(len(df_test)):
    # get the page and tables for this row
    page = take_url_return_page(df_test.iloc[i,9])
    def_table, punt_table = take_page_return_tables(page)

    # get the defensive stats and update the column names
    stat_df = get_stats(def_table, def_columns)
    if stat_df.empty:
      print('Mike')
    else:
      row_to_add = stat_df[stat_df['year'] == 'Career']
      prefix = str(row_to_add.loc[0, 'year']) + '_'
      updated_columns = [prefix + col for col in def_columns]
      row_to_add.columns = updated_columns

      # add the punt return information to the original DataFrame
      if punt_table == 0:
        df_test.at[i, 'returner'] = 0
      else:
        df_test.at[i, 'returner'] = 1

      # update the original row with the new Career row
      df_test.loc[i, updated_columns] = row_to_add.iloc[0]
      print(df_test.loc[i])
      
      subset_df = stat_df.iloc[:-1, :]
      row_to_add = None

      for j in range(len(subset_df)-1, -1, -1):
          row = subset_df.iloc[j]
          games_str = row['games']
          if games_str:
              games = int(games_str)
          else:
              games = 0 # or some other default value
          if games >= 6:
              row_to_add = row
              break

      if row_to_add is None:
          row_to_add = subset_df.iloc[-1]

      # add the string 'Best_' to all the column names in row_to_add
      prefix = 'Best_'
      updated_columns = [prefix + col for col in row_to_add.index]
      row_to_add.columns = updated_columns

      df_test.loc[i, updated_columns] = row_to_add.values
    time.sleep(7)
  return df_test


In [ ]:
df_def.iloc[0,9]

'http://www.sports-reference.com/cfb/players/travon-walker-1.html'

In [ ]:
page=take_url_return_page(df_def.iloc[0,9])

In [ ]:
print(page.prettify())



<!DOCTYPE html>
<html class="no-js" data-root="/home/cfb/build" data-version="klecko-" lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="ie=edge" http-equiv="x-ua-compatible"/>
  <meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0" name="viewport">
   <link href="https://cdn.ssref.net/req/202303021" rel="dns-prefetch"/>
   <!-- Quantcast Choice. Consent Manager Tag v2.0 (for TCF 2.0) -->
   <script async="true" type="text/javascript">
    (function() {
	var host = window.location.hostname;
	var element = document.createElement('script');
	var firstScript = document.getElementsByTagName('script')[0];
	var url = 'https://cmp.quantcast.com'
	    .concat('/choice/', 'XwNYEpNeFfhfr', '/', host, 
		    '/choice.js?tag_version=V2');
	var uspTries = 0;
	var uspTriesLimit = 3;
	element.async = true;
	element.type = 'text/javascript';
	element.src = url;
	
	firstScript.parentNode.insertBefore(element, firstScript);
	
	function makeStub() {
	    var TCF_LOCATOR_NAM

In [ ]:
def_table, punt_table = take_page_return_tables(page)

In [ ]:
def_table =page.find('table')

In [ ]:
def_table

In [ ]:
def_table.find_all('tr')

AttributeError: ignored

In [ ]:
page.find('table', {'id': 'defense'})


In [ ]:
import warnings

# Disable all warnings
warnings.filterwarnings("ignore")

# Running the Loop

In [29]:
df_def.query('year >= 2005 and year <= 2019')

,year,round,draft_pick,draft_team,player,player_url,position,age,college,college_stats_url,...,bday,hs,height,weight,height_decimal,birthplace_town,birthplace_state,home_state,conference,bmi
1544,2016,1,3,LAC,Joey Bosa,https://www.pro-football-reference.com/players...,DE,21.0,Ohio St.,http://www.sports-reference.com/cfb/players/jo...,...,7/11/1995,"St. Thomas Aquinas,FL",6'5,280.0,6.42,Miami,"FL,",FL,BIG10,33.2
1546,2016,1,5,JAX,Jalen Ramsey,https://www.pro-football-reference.com/players...,CB,21.0,Florida St.,http://www.sports-reference.com/cfb/players/ja...,...,10/24/1994,"Brentwood Academy,TN",6'1,208.0,6.08,Smyrna,"TN,",TN,ACC,27.5
1548,2016,1,7,SFO,DeForest Buckner,https://www.pro-football-reference.com/players...,DE,22.0,Oregon,http://www.sports-reference.com/cfb/players/de...,...,3/17/1994,"Punahou School,HI",6'7,295.0,6.58,Wainanae,"HI,",HI,PAC12,33.3
1550,2016,1,9,CHI,Leonard Floyd,https://www.pro-football-reference.com/players...,OLB,23.0,Georgia,http://www.sports-reference.com/cfb/players/le...,...,9/8/1992,"Hargrave Military Academy,VA,Dodge Co.,GA",6'5,240.0,6.42,Atlanta,"GA,",GA,SEC,28.4
1551,2016,1,10,NYG,Eli Apple,https://www.pro-football-reference.com/players...,CB,21.0,Ohio St.,http://www.sports-reference.com/cfb/players/el...,...,8/9/1995,"Eastern,NJ",6'1,199.0,6.08,Philadelphia,"PA,",PA,BIG10,26.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3321,2010,7,251,LVR,Stevie Brown,https://www.pro-football-reference.com/players...,DB,23.0,Michigan,http://www.sports-reference.com/cfb/players/st...,...,7/17/1987,"Columbus East,IN",6'0,213.0,6.00,Dallas,"TX,",TX,BIG10,28.9
3322,2010,7,252,MIA,Austin Spitler,https://www.pro-football-reference.com/players...,LB,23.0,Ohio St.,http://www.sports-reference.com/cfb/players/au...,...,10/26/1986,"Bellbrook,OH",6'3,244.0,6.25,Dayton,"OH,",OH,BIG10,30.5
3323,2010,7,253,TAM,Erik Lorig,https://www.pro-football-reference.com/players...,DE,23.0,Stanford,http://www.sports-reference.com/cfb/players/er...,...,11/17/1986,"Palos Verdes Peninsula,CA",6'4,265.0,6.33,Rolling Hills,"CA,",CA,PAC12,32.3
3324,2010,7,254,LAR,Josh Hull,https://www.pro-football-reference.com/players...,LB,23.0,Penn St.,http://www.sports-reference.com/cfb/players/jo...,...,5/21/1987,"Penns Valley,PA",6'2,236.0,6.17,State College,"PA,",PA,BIG10,30.3


In [65]:
#def_2000_4 = df_def[df_def['year']==2017]
def_1993_4 = df_def.query('year >= 1993 and year <= 1994')
def_1993_4 = def_1993_4.reset_index(drop=True)

In [66]:
results_1993_4 = the_loop(def_1993_4)

Mike
Mike


<ipython-input-15-ab3dcf27886a>:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([stat_list], columns=columns))
<ipython-input-15-ab3dcf27886a>:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([stat_list], columns=columns))
<ipython-input-15-ab3dcf27886a>:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([stat_list], columns=columns))
<ipython-input-15-ab3dcf27886a>:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([stat_list], columns=columns))


year                                                                     1994
round                                                                       1
draft_pick                                                                  8
draft_team                                                                SEA
player                                                              Sam Adams
player_url                  https://www.pro-football-reference.com/players...
position                                                                   DT
age                                                                      21.0
college                                                             Texas A&M
college_stats_url           http://www.sports-reference.com/cfb/players/sa...
pic_url                     https://www.pro-football-reference.com/req/201...
bday                                                                6/13/1973
hs                                                           Cyp

<ipython-input-15-ab3dcf27886a>:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([stat_list], columns=columns))
<ipython-input-15-ab3dcf27886a>:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([stat_list], columns=columns))
<ipython-input-15-ab3dcf27886a>:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([stat_list], columns=columns))
<ipython-input-15-ab3dcf27886a>:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([stat_list], columns=columns))
<ipython-input-15-ab3dcf27886a>:61: FutureWarning: The frame.append method is de

year                                 1994
round                                   1
draft_pick                              9
draft_team                            CLE
player                    Antonio Langham
                               ...       
Best_passes_defended                  NaN
Best_fumbles_recovered                NaN
Best_fumble_yards                     NaN
Best_fumble_td                        NaN
Best_forced_fumbles                   NaN
Name: 3, Length: 62, dtype: object
Mike
Mike
year                             1994
round                               1
draft_pick                         12
draft_team                        NYJ
player                    Aaron Glenn
                             ...     
Best_passes_defended              NaN
Best_fumbles_recovered            NaN
Best_fumble_yards                 NaN
Best_fumble_td                    NaN
Best_forced_fumbles               NaN
Name: 5, Length: 62, dtype: object


...

<ipython-input-15-ab3dcf27886a>:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([stat_list], columns=columns))
<ipython-input-15-ab3dcf27886a>:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([stat_list], columns=columns))


year                              1993
round                                7
draft_pick                         196
draft_team                         DAL
player                    Brock Marion
                              ...     
Best_passes_defended               NaN
Best_fumbles_recovered             NaN
Best_fumble_yards                  NaN
Best_fumble_td                     NaN
Best_forced_fumbles                NaN
Name: 91, Length: 62, dtype: object


<ipython-input-15-ab3dcf27886a>:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([stat_list], columns=columns))
<ipython-input-15-ab3dcf27886a>:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([stat_list], columns=columns))
<ipython-input-15-ab3dcf27886a>:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([stat_list], columns=columns))
<ipython-input-15-ab3dcf27886a>:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([stat_list], columns=columns))
<ipython-input-15-ab3dcf27886a>:61: FutureWarning: The frame.append method is de

year                                  1993
round                                    8
draft_pick                             201
draft_team                             DET
player                    Kevin Miniefield
                                ...       
Best_passes_defended                   NaN
Best_fumbles_recovered                 NaN
Best_fumble_yards                      NaN
Best_fumble_td                         NaN
Best_forced_fumbles                    NaN
Name: 92, Length: 62, dtype: object
Mike
year                               1993
round                                 8
draft_pick                          214
draft_team                          HOU
player                    Blaine Bishop
                              ...      
Best_passes_defended                NaN
Best_fumbles_recovered              NaN
Best_fumble_yards                   NaN
Best_fumble_td                      NaN
Best_forced_fumbles                 NaN
Name: 93, Length: 62, dtype: object


<ipython-input-15-ab3dcf27886a>:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([stat_list], columns=columns))
<ipython-input-15-ab3dcf27886a>:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([stat_list], columns=columns))
<ipython-input-15-ab3dcf27886a>:61: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([stat_list], columns=columns))


In [57]:
results_2000_4.head(20)

,year,round,draft_pick,draft_team,player,player_url,position,age,college,college_stats_url,...,Best_sacks,Best_interceptions,Best_int_yards,Best_int_yards_per_int,Best_int_td,Best_passes_defended,Best_fumbles_recovered,Best_fumble_yards,Best_fumble_td,Best_forced_fumbles
0,2004,1,5,WAS,Sean Taylor,https://www.pro-football-reference.com/players...,DB,21.0,Miami (FL),http://www.sports-reference.com/cfb/players/se...,...,,10,184,18.4,3,,,,,
1,2004,1,8,ATL,DeAngelo Hall,https://www.pro-football-reference.com/players...,DB,20.0,Virginia Tech,http://www.sports-reference.com/cfb/players/de...,...,,1,0,0.0,0,,,28,1,
2,2004,1,10,HOU,Dunta Robinson,https://www.pro-football-reference.com/players...,DB,22.0,South Carolina,http://www.sports-reference.com/cfb/players/du...,...,,1,29,29.0,1,,,,,
3,2004,1,12,NYJ,Jonathan Vilma,https://www.pro-football-reference.com/players...,LB,22.0,Miami (FL),http://www.sports-reference.com/cfb/players/jo...,...,,0,0,,0,,1,2,1,
4,2004,1,14,CHI,Tommie Harris,https://www.pro-football-reference.com/players...,DT,21.0,Oklahoma,http://www.sports-reference.com/cfb/players/to...,...,,,,,,,1,14,0,
5,2004,1,17,DEN,D.J. Williams,https://www.pro-football-reference.com/players...,LB,22.0,Miami (FL),http://www.sports-reference.com/cfb/players/dj...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2004,1,18,NOR,Will Smith,https://www.pro-football-reference.com/players...,DE,23.0,Ohio St.,http://www.sports-reference.com/cfb/players/wi...,...,,1,0,0.0,0,,,,,
7,2004,1,20,MIN,Kenechi Udeze,https://www.pro-football-reference.com/players...,DE,21.0,USC,http://www.sports-reference.com/cfb/players/ke...,...,,0,0,,0,,1,0,1,
8,2004,1,21,NWE,Vince Wilfork,https://www.pro-football-reference.com/players...,DT,22.0,Miami (FL),http://www.sports-reference.com/cfb/players/vi...,...,,,,,,,1,0,0,
9,2004,1,25,GNB,Ahmad Carroll,https://www.pro-football-reference.com/players...,DB,21.0,Arkansas,http://www.sports-reference.com/cfb/players/ah...,...,,3,0,0.0,0,,,,,


In [67]:
results_1993_4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94 entries, 0 to 93
Data columns (total 62 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   year                      94 non-null     int64  
 1   round                     94 non-null     int64  
 2   draft_pick                94 non-null     int64  
 3   draft_team                94 non-null     object 
 4   player                    94 non-null     object 
 5   player_url                85 non-null     object 
 6   position                  94 non-null     object 
 7   age                       85 non-null     float64
 8   college                   94 non-null     object 
 9   college_stats_url         94 non-null     object 
 10  pic_url                   22 non-null     object 
 11  bday                      85 non-null     object 
 12  hs                        85 non-null     object 
 13  height                    85 non-null     object 
 14  weight      

In [68]:
results_1993_4.to_csv('def_stats_1993_4.csv')

In [ ]:
results_2021.sample(20)

,year,round,draft_pick,draft_team,player,player_url,position,age,college,college_stats_url,...,Best_sacks,Best_interceptions,Best_int_yards,Best_int_yards_per_int,Best_int_td,Best_passes_defended,Best_fumbles_recovered,Best_fumble_yards,Best_fumble_td,Best_forced_fumbles
13,2021,1,32,TAM,Joe Tryon-Shoyinka,https://www.pro-football-reference.com/players...,OLB,22.0,Washington,http://www.sports-reference.com/cfb/players/jo...,...,8.0,0,0,,0,1,0,,,0
73,2021,5,164,DEN,Jamar Johnson,https://www.pro-football-reference.com/players...,DB,21.0,Indiana,http://www.sports-reference.com/cfb/players/ja...,...,1.0,4,94,23.5,0,4,0,,,1
62,2021,4,136,ARI,Marco Wilson,https://www.pro-football-reference.com/players...,CB,22.0,Florida,http://www.sports-reference.com/cfb/players/ma...,...,0.0,0,0,,0,4,1,,,0
8,2021,1,26,CLE,Greg Newsome II,https://www.pro-football-reference.com/players...,DB,21.0,Northwestern,http://www.sports-reference.com/cfb/players/gr...,...,0.0,0,0,,0,9,1,,,0
4,2021,1,18,MIA,Jaelan Phillips,https://www.pro-football-reference.com/players...,DL,22.0,Miami (FL),http://www.sports-reference.com/cfb/players/ja...,...,8.0,1,0,0.0,0,3,0,,,0
68,2021,5,152,DEN,Caden Sterns,https://www.pro-football-reference.com/players...,DB,21.0,Texas,http://www.sports-reference.com/cfb/players/ca...,...,0.0,1,0,0.0,0,3,0,,,0
106,2021,6,228,CHI,Thomas Graham,https://www.pro-football-reference.com/players...,CB,22.0,Oregon,http://www.sports-reference.com/cfb/players/th...,...,0.0,2,26,13.0,0,10,0,,,1
46,2021,3,104,BAL,Brandon Stephens,https://www.pro-football-reference.com/players...,CB,23.0,SMU,http://www.sports-reference.com/cfb/players/br...,...,0.0,1,0,0.0,0,10,1,,,0
75,2021,5,166,CAR,Keith Taylor,https://www.pro-football-reference.com/players...,DB,22.0,Washington,http://www.sports-reference.com/cfb/players/ke...,...,0.0,0,0,,0,5,0,,,0
91,2021,6,189,PHI,Marlon Tuipulotu,https://www.pro-football-reference.com/players...,DL,22.0,USC,http://www.sports-reference.com/cfb/players/ma...,...,2.0,0,0,,0,3,0,,,1


In [ ]:
df_test

,year,round,draft_pick,draft_team,player,player_url,position,age,college,college_stats_url,...,bday,hs,height,weight,height_decimal,birthplace_town,birthplace_state,home_state,conference,bmi
0,2022,1,1,JAX,Travon Walker,https://www.pro-football-reference.com/players...,DE,21.0,Georgia,http://www.sports-reference.com/cfb/players/tr...,...,12/18/2000,"Upson,GA",6'5,275.0,6.42,Thomaston,"GA,",GA,SEC,32.6
1,2022,1,2,DET,Aidan Hutchinson,https://www.pro-football-reference.com/players...,DE,22.0,Michigan,http://www.sports-reference.com/cfb/players/ai...,...,8/9/2000,"Divine Child,MI",6'7,265.0,6.58,MI,NaN,MI,BIG10,29.9
2,2022,1,3,HOU,Derek Stingley Jr.,https://www.pro-football-reference.com/players...,CB,21.0,LSU,http://www.sports-reference.com/cfb/players/de...,...,6/20/2001,"Dunham,LA",6'1,195.0,6.08,Baton Rouge,"LA,",LA,SEC,25.8
3,2022,1,4,NYJ,Sauce Gardner,https://www.pro-football-reference.com/players...,CB,22.0,Cincinnati,http://www.sports-reference.com/cfb/players/ah...,...,8/31/2000,"Martin Luther King,MI",6'3,200.0,6.25,Detroit,"MI,",MI,AAC,25.0
4,2022,1,5,NYG,Kayvon Thibodeaux,https://www.pro-football-reference.com/players...,DE,21.0,Oregon,http://www.sports-reference.com/cfb/players/ka...,...,12/15/2000,"Susan Miller Dorsey,CA,Oaks Christian,CA",6'5,250.0,6.42,CA,NaN,CA,PAC12,29.6
5,2022,1,13,PHI,Jordan Davis,https://www.pro-football-reference.com/players...,DT,22.0,Georgia,http://www.sports-reference.com/cfb/players/jo...,...,1/12/2000,"Mallard Creek,NC",6'6,340.0,6.50,NaN,NaN,NC,SEC,39.3
6,2022,1,21,KAN,Trent McDuffie,https://www.pro-football-reference.com/players...,CB,21.0,Washington,http://www.sports-reference.com/cfb/players/tr...,...,9/13/2000,"Mater Dei,CA,Servite,CA,St. John Bosco,CA",5'11,195.0,5.92,CA,NaN,CA,PAC12,27.2
7,2022,1,22,GNB,Quay Walker,https://www.pro-football-reference.com/players...,LB,22.0,Georgia,http://www.sports-reference.com/cfb/players/qu...,...,5/8/2000,"Crisp County,GA",6'4,240.0,6.33,NaN,NaN,GA,SEC,29.2
8,2022,1,23,BUF,Kaiir Elam,https://www.pro-football-reference.com/players...,CB,21.0,Florida,http://www.sports-reference.com/cfb/players/ka...,...,5/5/2001,"Benjamin School,FL",6'2,195.0,6.17,NaN,NaN,FL,SEC,25.0
9,2022,1,26,NYJ,Jermaine Johnson II,https://www.pro-football-reference.com/players...,LB,23.0,Florida St.,http://www.sports-reference.com/cfb/players/je...,...,1/8/1999,"Eden Prairie,MN",6'5,260.0,6.42,NaN,NaN,MN,ACC,30.8


# Testing Zone

In [ ]:
def_2019 = def_2019[def_2019['draft_pick']==230]

In [ ]:
mike_df=pd.DataFrame()

In [ ]:
mike_df

""


In [ ]:
dave_df = results_2021.iloc[0:3,]

In [ ]:
dave_df

,year,round,draft_pick,draft_team,player,player_url,position,age,college,college_stats_url,...,Best_sacks,Best_interceptions,Best_int_yards,Best_int_yards_per_int,Best_int_td,Best_passes_defended,Best_fumbles_recovered,Best_fumble_yards,Best_fumble_td,Best_forced_fumbles
0,2021,1,8,CAR,Jaycee Horn,https://www.pro-football-reference.com/players...,DB,21.0,South Carolina,http://www.sports-reference.com/cfb/players/ja...,...,0.0,2,34,17.0,0,6,0,,,0
1,2021,1,9,DEN,Patrick Surtain II,https://www.pro-football-reference.com/players...,DB,21.0,Alabama,http://www.sports-reference.com/cfb/players/pa...,...,0.0,1,25,25.0,1,9,0,,,0
2,2021,1,12,DAL,Micah Parsons,https://www.pro-football-reference.com/players...,LB,22.0,Penn St.,http://www.sports-reference.com/cfb/players/mi...,...,5.0,0,0,,0,5,1,,,4


In [ ]:
df_test2 = df_test.iloc[0:2]

In [ ]:
dave_df.append(dave_df.iloc[0:1,])

<ipython-input-84-437372abbd5e>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dave_df.append(dave_df.iloc[0:1,])


,year,round,draft_pick,draft_team,player,player_url,position,age,college,college_stats_url,...,Best_sacks,Best_interceptions,Best_int_yards,Best_int_yards_per_int,Best_int_td,Best_passes_defended,Best_fumbles_recovered,Best_fumble_yards,Best_fumble_td,Best_forced_fumbles
0,2021,1,8,CAR,Jaycee Horn,https://www.pro-football-reference.com/players...,DB,21.0,South Carolina,http://www.sports-reference.com/cfb/players/ja...,...,0.0,2,34,17.0,0,6,0,,,0
1,2021,1,9,DEN,Patrick Surtain II,https://www.pro-football-reference.com/players...,DB,21.0,Alabama,http://www.sports-reference.com/cfb/players/pa...,...,0.0,1,25,25.0,1,9,0,,,0
2,2021,1,12,DAL,Micah Parsons,https://www.pro-football-reference.com/players...,LB,22.0,Penn St.,http://www.sports-reference.com/cfb/players/mi...,...,5.0,0,0,,0,5,1,,,4
0,2021,1,8,CAR,Jaycee Horn,https://www.pro-football-reference.com/players...,DB,21.0,South Carolina,http://www.sports-reference.com/cfb/players/ja...,...,0.0,2,34,17.0,0,6,0,,,0


In [ ]:
stat_test = stat_df.iloc[0:1]

In [ ]:
blank = pd.DataFrame()

In [ ]:
if blank.empty:
  print('Mike')

Mike


# The Curious Case of Greg Stroman
Greg lists as wr, db, cb

his first table is punt returns, THEN defense

https://www.sports-reference.com/cfb/players/greg-stroman-1.html

we get a IndexError: list index out of range

for the line:
punt_ret = all_punt_ret(text=lambda text: isinstance(text, Comment))[0]

In [ ]:
greg = def_2018[def_2018['draft_pick']==241]

In [ ]:
url = 'https://www.sports-reference.com/cfb/players/greg-stroman-1.html'

In [ ]:
page = take_url_return_page(url)

In [ ]:
page.find('table')

## Redefining Function: take_page_return_tables

In [44]:
def take_page_return_tables1(page):
  # this function takes a soup page and looks for the table id defense and returns it.  same for the punt_ret table (which is hidden in the comments)
  def_table = page.find('table', {'id': 'defense'})
  # finding where punt return info is located
  all_punt_ret = page.find(id="all_punt_ret")
  # adding an exception.  If all_punt_ret is empty, we return 0
  if all_punt_ret is None:
    return def_table, 0
  else:
    # getting the info inside the comments
    try:
      punt_ret = all_punt_ret(text=lambda text: isinstance(text, Comment))[0]
      # making the data parsable by BS
      punt_soup=BeautifulSoup(punt_ret, 'html.parser')
      punt_table = punt_soup.find('table', {'id': 'punt_ret'})
      return def_table, punt_table
    except IndexError:
      def_table, punt_table = greg_stroman(page)
      return def_table, punt_table

In [41]:
def greg_stroman(page):
  print('Mike')
  punt_table = page.find('table', {'id': 'punt_ret'})
  # pacman jones clause (aka when punt ret is listed as kick return)
  if punt_table is None:
    punt_table = page.find('table', {'id': 'kick_ret'})
  all_defense = page.find(id='all_defense')
  if all_defense is None:
    def_table = 0
  else:
    defense = all_defense(text=lambda text: isinstance(text, Comment))[0]
    def_soup = BeautifulSoup(defense, 'html.parser')
    def_table = def_soup.find('table', {'id': 'defense'})
 
  return def_table, punt_table
     

In [ ]:
def_table, punt_table = take_page_return_tables2(page)

In [ ]:
def_table

<table class="sortable stats_table" data-cols-to-freeze=",1" id="defense">
<caption>Defense &amp; Fumbles Table</caption>
<colgroup><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/></colgroup>
<thead>
<tr class="over_header">
<th aria-label="" class="over_header center" colspan="5" data-stat=""></th><th></th>
<th aria-label="" class="over_header center" colspan="5" data-stat="header_tackles">Tackles</th>
<th aria-label="" class="over_header center" colspan="5" data-stat="header_def_int">Def Int</th>
<th aria-label="" class="over_header center" colspan="4" data-stat="header_fumbles">Fumbles</th>
</tr>
<tr>
<th aria-label="Year" class="poptip sort_default_asc right" data-stat="year_id" scope="col">Year</th>
<th aria-label="School" class="poptip sort_default_asc left" data-stat="school_name" data-tip="School Name" scope="col">School</th>
<th aria-label="Conf" class="poptip sort_default_asc left" data-stat="conf_abbr" s

In [ ]:
def_table = page.find('table', {'id': 'defense'})
type(def_table)

NoneType

In [ ]:
if def_table is None:
  print('Mike')

Mike


In [ ]:
all_punt_ret = page.find(id="all_punt_ret")

In [ ]:
all_punt_ret

<div class="table_wrapper" id="all_punt_ret">
<div class="section_heading assoc_punt_ret" id="punt_ret_sh">
<span class="section_anchor" data-label="Punt &amp; Kick Returns" id="punt_ret_link"></span><h2>Punt &amp; Kick Returns</h2> <div class="section_heading_text">
<ul><li>* indicates bowl stats included</li>
</ul>
</div>
</div>
<div class="table_container" id="div_punt_ret">
<table class="sortable stats_table" data-cols-to-freeze=",1" id="punt_ret">
<caption>Punt &amp; Kick Returns Table</caption>
<colgroup><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/></colgroup>
<thead>
<tr class="over_header">
<th aria-label="" class="over_header center" colspan="5" data-stat=""></th><th></th>
<th aria-label="" class="over_header center" colspan="4" data-stat="header_punt_returns">Punt Ret</th>
<th aria-label="" class="over_header center" colspan="4" data-stat="header_kick_returns">Kick Ret</th>
</tr>
<tr>
<th aria-label="Year" class="poptip sort_default_asc 

In [ ]:
print(page.prettify())

<!DOCTYPE html>
<html class="no-js" data-root="/home/cfb/build" data-version="klecko-" lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="ie=edge" http-equiv="x-ua-compatible"/>
  <meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0" name="viewport">
   <link href="https://cdn.ssref.net/req/202303021" rel="dns-prefetch"/>
   <!-- Quantcast Choice. Consent Manager Tag v2.0 (for TCF 2.0) -->
   <script async="true" type="text/javascript">
    (function() {
	var host = window.location.hostname;
	var element = document.createElement('script');
	var firstScript = document.getElementsByTagName('script')[0];
	var url = 'https://cmp.quantcast.com'
	    .concat('/choice/', 'XwNYEpNeFfhfr', '/', host, 
		    '/choice.js?tag_version=V2');
	var uspTries = 0;
	var uspTriesLimit = 3;
	element.async = true;
	element.type = 'text/javascript';
	element.src = url;
	
	firstScript.parentNode.insertBefore(element, firstScript);
	
	function makeStub() {
	    var TCF_LOCATOR_NAM

In [ ]:
all_defense = page.find(id='all_defense')

In [ ]:
all_defense

<div class="table_wrapper setup_commented commented" id="all_defense">
<div class="section_heading assoc_defense" id="defense_sh">
<span class="section_anchor" data-label="Defense &amp; Fumbles" id="defense_link"></span><h2>Defense &amp; Fumbles</h2> <div class="section_heading_text">
<ul><li>* indicates bowl stats included</li>
</ul>
</div>
</div><div class="placeholder"></div>
<!--

<div class="table_container" id="div_defense">
    
    <table class="sortable stats_table" id="defense" data-cols-to-freeze=",1">
    <caption>Defense &amp; Fumbles Table</caption>
    

   <colgroup><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col></colgroup>
   <thead>

      
      <tr class="over_header">
         <th aria-label="" data-stat="" colspan="5" class=" over_header center" ></th><th></th>
         <th aria-label="" data-stat="header_tackles" colspan="5" class=" over_header center" >Tackles</th>
         <th aria-label="" data-stat="header_

# PACman Jones
https://www.sports-reference.com/cfb/players/adam-jones-1.html

In [26]:
url = 'https://www.sports-reference.com/cfb/players/adam-jones-1.html'

In [27]:
page = take_url_return_page(url)

In [28]:
all_defense = page.find(id='all_defense')

In [ ]:
print(page.prettify())

In [29]:
all_defense

<div class="table_wrapper setup_commented commented" id="all_defense">
<div class="section_heading assoc_defense" id="defense_sh">
<span class="section_anchor" data-label="Defense &amp; Fumbles" id="defense_link"></span><h2>Defense &amp; Fumbles</h2> <div class="section_heading_text">
<ul><li>* indicates bowl stats included</li>
</ul>
</div>
</div><div class="placeholder"></div>
<!--

<div class="table_container" id="div_defense">
    
    <table class="sortable stats_table" id="defense" data-cols-to-freeze=",1">
    <caption>Defense &amp; Fumbles Table</caption>
    

   <colgroup><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col><col></colgroup>
   <thead>

      
      <tr class="over_header">
         <th aria-label="" data-stat="" colspan="5" class=" over_header center" ></th><th></th>
         <th aria-label="" data-stat="header_tackles" colspan="5" class=" over_header center" >Tackles</th>
         <th aria-label="" data-stat="header_

In [30]:
def_table, punt_table = greg_stroman(page)

In [31]:
def_table

<table class="sortable stats_table" data-cols-to-freeze=",1" id="defense">
<caption>Defense &amp; Fumbles Table</caption>
<colgroup><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/><col/></colgroup>
<thead>
<tr class="over_header">
<th aria-label="" class="over_header center" colspan="5" data-stat=""></th><th></th>
<th aria-label="" class="over_header center" colspan="5" data-stat="header_tackles">Tackles</th>
<th aria-label="" class="over_header center" colspan="5" data-stat="header_def_int">Def Int</th>
<th aria-label="" class="over_header center" colspan="4" data-stat="header_fumbles">Fumbles</th>
</tr>
<tr>
<th aria-label="Year" class="poptip sort_default_asc right" data-stat="year_id" scope="col">Year</th>
<th aria-label="School" class="poptip sort_default_asc left" data-stat="school_name" data-tip="School Name" scope="col">School</th>
<th aria-label="Conf" class="poptip sort_default_asc left" data-stat="conf_abbr" s

In [35]:
def_table, punt_table = take_page_return_tables(page)

In [36]:
punt_table

0

In [33]:
def_table

In [37]:
all_punt_ret = page.find(id="all_punt_ret")

In [38]:
all_punt_ret

# Redefining Fuction: take_page_return_tables

In [47]:
def take_page_return_tables(page):
  # this function takes a soup page and looks for the table id defense and returns it.  same for the punt_ret table (which is hidden in the comments)
  def_table = page.find('table', {'id': 'defense'})
  # finding where punt return info is located
  all_punt_ret = page.find(id="all_punt_ret")
  # adding an exception.  If all_punt_ret is empty, we return 0
  if all_punt_ret is None:
    all_punt_ret = page.find(id = 'all_kick_ret')
  if all_punt_ret is None:
    return def_table, 0
  else:
    # getting the info inside the comments
    try:
      punt_ret = all_punt_ret(text=lambda text: isinstance(text, Comment))[0]
      # making the data parsable by BS
      punt_soup=BeautifulSoup(punt_ret, 'html.parser')
      punt_table = punt_soup.find('table', {'id': 'punt_ret'})
      return def_table, punt_table
    except IndexError:
      def_table, punt_table = greg_stroman(page)
      return def_table, punt_table

In [77]:
table = page.find('table', {'class': 'sortable stats_table'})

In [78]:
table.get('id')

'kick_ret'